In [ ]:
from numpy.random import test
from keras.datasets import boston_housing
from keras import models
from keras import layers
import numpy as np
import matplotlib.pyplot as plt


(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

# print(train_data.shape)
# print(test_data.shape)
# print(train_targets)

mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    
    model.compile(
        optimizer='rmsprop',
        loss='mse',
        metrics=['mae']
    )
    return model
  
k = 4
num_val_samples = len(train_data) // k
num_epochs = 500
# all_scores = []
all_mae_histories = []

for i in range(k):
    print('processing fold #', i)
    val_data = train_data[
        i * num_val_samples: (i + 1) * num_val_samples
    ]
    val_targets = train_targets[
        i * num_val_samples: (i + 1) * num_val_samples
    ]

    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
         axis=0
    )
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
         axis=0
    )

    model = build_model()
    history = model.fit(
        partial_train_data,
        partial_train_targets,
        validation_data=(val_data, val_targets),
        epochs=num_epochs,
        batch_size=1,
        verbose=0
    )
    val_mse, val_mae = model.evaluate(
        val_data,
        val_targets,
        verbose=0
    )
    # for key, value in history.history.items() :
    #     print (key, value)
    mae_history = history.history['mae']
    all_mae_histories.append(mae_history)

average_mae_history = [
    np.mean([x[i] for x in all_mae_histories]) for i in range (num_epochs)
]

def smooth_curve(points, factor=0.9):
    smoothed_points=[]
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points


smoothed_mae_history = smooth_curve(average_mae_history[10:])

plt.plot(range(1, len(smoothed_mae_history) + 1), smoothed_mae_history)
plt.xlabel('Эпохи')
plt.ylabel('Оценка MAE')
plt.show()

model = build_model()

model.fit(
        train_data,
        train_targets,
        epochs=80,
        batch_size=16,
        verbose=0
  )

test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

print(test_mse_score, test_mae_score)